In [31]:
import yaml, subprocess, os, json, sys
from datetime import datetime
import pandas as pd
import numpy as np
from dotenv import dotenv_values
import openstack
env = dotenv_values("cyb.env")

def create_connection(auth_url, region, project_name, username, password,
                      user_domain, project_domain, project_id):
    return openstack.connect(
        auth_url=auth_url,
        project_name=project_name,
        username=username,
        password=password,
        region_name=region,
        user_domain_name=user_domain,
        project_domain_name=project_domain,
        project_id=project_id
    )
    
def collect_info(WATCHDOG_ADDRESS, key):
    conn = create_connection(auth_url=env['OS_AUTH_URL'], region=env['OS_REGION_NAME'],
    project_name=env['OS_PROJECT_NAME'], username=env['OS_USERNAME'],
    password=env['OS_PASSWORD'], user_domain=env['OS_USER_DOMAIN_NAME'],
    project_domain=env['OS_PROJECT_DOMAIN_NAME'], project_id=env['OS_PROJECT_ID'])
    
    subprocess.run(['scp', '-i', key, "-o", "StrictHostKeyChecking=no", "get_nodeinfo.py",
    "ubuntu@{}:/home/ubuntu/".format(WATCHDOG_ADDRESS)])

    COMMAND = 'python3 get_nodeinfo.py'
    subprocess.Popen(["ssh", "-i", key, 
                        "-o", "StrictHostKeyChecking=no", "ubuntu@%s" % WATCHDOG_ADDRESS, COMMAND],
                        shell=False,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE)
    subprocess.run(['sleep', '3'])
    subprocess.run(['scp', '-i', key, "-o", "StrictHostKeyChecking=no", 
        "ubuntu@{}:~/nodeinfo.json".format(WATCHDOG_ADDRESS), os.getcwd()])

    subprocess.run(['sleep', '2'])

    with open('nodeinfo.json', 'r') as f:
        data = json.load(f)
        rows = data['nodeinfo']

    for row in rows:
        instance_id = conn.compute.find_server(row[0]).id
        host_id = conn.compute.get_server(instance_id).host_id
        row.append(instance_id)
        row.append(host_id)
    df = pd.DataFrame(np.array(rows),
        columns=['NodeName', 'IP', 'NodeAddress', 'IsValidator', 'InstanceID', 'HostID'])
    # sort dataframe
    df['Index'] = [int(name.split('-')[1]) for name in df.NodeName]
    df = df.set_index(keys=df.Index).drop(labels='Index', axis=1).sort_index()
    
    with open('nodeinfo.json', 'w') as f:
        json.dump({'nodeinfo': rows}, f, indent=4)
    return df

def run(WATCHDOG_ADDRESS, key, SEND_RATES):
    df = collect_info(WATCHDOG_ADDRESS, key)

    DEFAULT_IP = df.IP.values[0]
    connection_url = "ws://" + DEFAULT_IP + ":8546"
    # if len(sys.argv) > 1:
    #     connection_url = "ws://" + sys.argv[1] + ":8546"
    networkconfig = 'networks/4node-ibft2/networkconfig.json'
    with open(networkconfig, 'r') as f:
        data = json.load(f)
        data['ethereum']['url'] = connection_url
    with open(networkconfig, 'w') as f:
        json.dump(data, f, indent=4)

    timestampStr = datetime.now().strftime("%Y%m%d-%H%M%S")
    directory = 'reports/' + timestampStr
    path = os.path.join(os.getcwd(), directory)
    os.mkdir(path)

    replicas = 5 # test replicas for each send rate
    rounds = 3 # test rounds: open, query and transfer
    for tps in SEND_RATES:
        with open('benchmarks/scenario/simple/config.yaml','r') as f:
            y=yaml.safe_load(f)
            for i in range(rounds):
                y['test']['rounds'][i]['rateControl']['opts']['tps'] = tps
        with open('benchmarks/scenario/simple/config.yaml','w') as f:
            yaml.dump(y,f,default_flow_style=False, sort_keys=False)

        for i in range(replicas):
            subprocess.run(['docker-compose', 'up'])
            subprocess.run(['cp', 'report.html', '{}/report-{}-{}.html'.format(directory, tps, i+1)])
            subprocess.run(['sleep', '10'])

    print(df)

    subprocess.run(['sleep', '10'])

    for _, row in df.iterrows():
        COMMAND = 'docker logs $(docker ps -q) > {}.log'.format(row['NodeName'])
        subprocess.Popen(["ssh", "-i", key, 
                            "-o", "StrictHostKeyChecking=no", "ubuntu@%s" % row['IP'], COMMAND],
                            shell=False,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

    subprocess.run(['sleep', '10'])

    collect_log(df, key)

def collect_log(df, key):
    timestampStr = datetime.now().strftime("%Y%m%d-%H%M%S")
    directory = '../data/logs-' + timestampStr
    log_path = os.path.join(os.getcwd(), directory)
    os.mkdir(log_path)
    # collect besu logs
    for _, row in df.iterrows():
        subprocess.run(['scp', '-i', key, "-o", "StrictHostKeyChecking=no", 
        "ubuntu@{}:/home/ubuntu/{}.log".format(row['IP'], row.NodeName), log_path])
    # collect caliper logs
    subprocess.run(['mv', 'caliper.log', log_path])
    # collect node info json data
    subprocess.run(['mv', 'nodeinfo.json', log_path])
    

# watchdogAddress = "192.168.226.176"
watchdogAddress = "10.2.12.61"
keyFile = "../data/rrg-bpet"
# keyFile = "../data/bpet.pem"
current_directory = os.getcwd()
sshKey = os.path.join(current_directory, keyFile)
sendRates = [50, 100, 150, 200, 250]
df = collect_info(watchdogAddress, sshKey)
df

,NodeName,IP,NodeAddress,IsValidator,InstanceID,HostID
Index,,,,,,
1,besu-1,10.2.1.9,0x781345fa5477b9a0d085ee5734ffa965bc3da068,True,40d368a4-75e5-4cd0-b06f-372561d0e89f,85ae958da561f450ef37abc13d5632a395bcb165544e8b...
2,besu-2,10.2.13.57,0xfff7774107f7c66c45321dd9b86ad09f21473c95,True,242f070e-879c-4f66-9d0f-e6338e751c84,12e678d36d6dc54feee7fb00fe48678bab6fd26a76a20d...
3,besu-3,10.2.9.234,0x709f53e59f541cc76c678dccba272817f450a0f9,True,b707c807-e0f1-4cdd-8ad1-10b4567f95e7,5bc3c72bf1b42515cb454e42d4bc94b7b7acd3f12519d0...
4,besu-4,10.2.13.45,0xde0ec9699a7d399c6d4d8b8d83d2c4f4a2f9efcd,True,6759a953-ac27-40f2-9c39-79c1078ddae3,cb73870ddbc9988c521cf95dc8522e821a4e3a9bf27da5...


In [10]:
import json
with open('nodeinfo.json') as f:
    data = json.load(f)
    rows = data['nodeinfo']
for row in rows:
    instance_id = conn.compute.find_server(row[0]).id
    host_id = conn.compute.get_server(instance_id).host_id
    row.append(instance_id)
    row.append(host_id)
with open('nodeinfo.json', 'w') as f:
    json.dump({"nodeinfo": rows}, f, indent=4)

In [7]:
import json
import pandas as pd
import numpy as np

with open('nodeinfo.json') as f:
    data = json.load(f)
    rows = data['nodeinfo']
df = pd.DataFrame(np.array(rows),
    columns=['NodeName', 'IP', 'NodeAddress', 'IsValidator'])
# sort dataframe
df['Index'] = [int(name.split('-')[1]) for name in df.NodeName]
df = df.set_index(keys=df.Index).drop(labels='Index', axis=1).sort_index()
df

,NodeName,IP,NodeAddress,IsValidator
Index,,,,
1,besu-1,10.2.12.215,0x381cc8a30236faa4e7716235b9478db116dcff61,True
2,besu-2,10.2.7.244,0x8816e57fe8fe788d276c8b6bd5422c9c7e5ffe74,True
3,besu-3,10.2.8.94,0xdd40c20241161b082ee4d916cd5685bd21f176d0,True
4,besu-4,10.2.12.202,0x7ba01799030b1a92b079a64dad7a444458f37c5a,True
5,besu-5,10.2.8.91,0x4fe188ada84c5611c4cf685e1fd98336b9b5cc9d,False


In [30]:
import yaml
def setup_monitors(df):
    benchconfig = 'benchmarks/scenario/simple/config.yaml'
    with open(benchconfig,'r') as f:
        y=yaml.safe_load(f)
        y['monitors']['resource'][0]['options']['containers'] = []
        for _, row in df.iterrows():
            container = 'http://' + row.IP +':2375/' + row.NodeName
            # y['monitors']['resource'][0]['options']['containers'] = []
            y['monitors']['resource'][0]['options']['containers'].append(container)
    with open(benchconfig, 'w') as f:
        yaml.dump(y, f, default_flow_style=False, sort_keys=False, indent=4)
setup_monitors(df)

In [3]:
import datetime 
now = datetime.datetime.now()
now.isoformat("T") + "Z"

'2022-02-24T10:00:47.858597Z'